# Google Scraper    
    
    Anish Work:
- Made a google scraper from scrath after amazon scraper


    google_url  = (https://www.google.com/shopping/product/7008326304182003253/reviews?q=google%20home&biw=1280&bih=639&prds=paur:ClkAsKraX5bIBh9mxyx3eoP5_ThWVQXaMDpCgoQ5o1a11DDxlM5RPgqnmQsmVBdPiKv-HORNf_a7b54ZmKDJa3j5-DfFORlGw1qvqVbWEhyV5ApvXZQpQ6CxzRIZAFPVH706XZwfVPbDmxlbW4Tngxvbu1o2NA,rstart:10&sa=X&ved=0ahUKEwj0v9jPtvXfAhUR2FQKHcD7AAMQqSQI_wE)
    
   product/........../ is the gsin code & putting the gsin code of the product you want will scrape that product
   
   change rstart:{_pgnumber_}0 for whatever page you want
   
   
## Need to do:


   
## How to use:

- Go to the function ReadGsin and add your gsin code to the list.

    (In the url example above it is the ...7008326304182003253...)

- Change pg_amounts to how many pages you want to iterate over

- After running the kernel the scraped data will be sent to google.json



In [ ]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep
import pandas as pd

In [9]:
def ParseReviews(gsin,pgnumber):
    # This script has only been tested with Amazon.com
    
    google_url  = ('https://www.google.com/shopping/product/{0}/reviews?q=google%20home&biw=1280&bih=639&prds=paur:ClkAsKraX8Po_BNHjxH-Yuazffm_I6ZTl-XFq7J9Cju-IhsWprvvBNQVbAtjPnj9S-0c283EL4MEVkciNKzFUhQXZKcjVzNzGHrLZrnO3-VSZ8ZUtWFeHIPJ5RIZAFPVH70GN1g15WffBXvfhjuFbXBgEL2GyQ,rstart:{1}0&sa=X&ved=0ahUKEwiz2_iUuPXfAhWHhlQKHTSKCMUQqSQI-gE'
    .format(gsin,pgnumber)) #gsin is unique google id
    
    # Add some recent user agent to prevent amazon from blocking the request 
    # Find some chrome user agent strings  here https://udger.com/resources/ua-list/browser-detail?browser=Chrome
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
    for i in range(5):
        response = get(google_url, headers = headers, verify=False, timeout=30)
        if response.status_code == 404:
            return {"url": amazon_url, "error": "page not found"}
        if response.status_code != 200:
            continue
        
        # Removing the null bytes from the response.
        cleaned_response = response.text.replace('\x00', '')
        
        parser = html.fromstring(cleaned_response)
        XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
        XPATH_REVIEW_SECTION_1 = './/div[@class="ESCbSb"]'
        XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'
        XPATH_AGGREGATE_RATING = '//div[@class="A85tye tIrLid"]/@aria-label'
        XPATH_PRODUCT_NAME = '//h1[@id="product-name"]//text()'
        XPATH_PRICE = './/span[@class="price"]'

        raw_product_price = parser.xpath(XPATH_PRICE)[0].text_content()
        raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)[0]
        raw_agg_rating = parser.xpath(XPATH_AGGREGATE_RATING)
        
        agg_rating = []
        
        for i in raw_agg_rating:
            agg_rating.append(i.split(' '))
            
        ratings_dict = {}
        for j in agg_rating:
            rating_key = j[1]
            rating_value = j[0]
            ratings_dict.update({rating_key: rating_value})
        reviews = parser.xpath(XPATH_REVIEW_SECTION_1)

        product_price = ''.join(raw_product_price).replace(',', '')
        product_name = ''.join(raw_product_name).strip()

        if not reviews:
            reviews = parser.xpath(XPATH_REVIEW_SECTION_2)
        reviews_list = []
        
        # Parsing individual reviews
        for review in reviews:
            XPATH_RATING  = './/div[@class="vq3ore"]/@aria-label'
            XPATH_REVIEW_HEADER = './/span[@class="tukK6e"]//text()'
            XPATH_REVIEW_POSTED_DATE = './/div[@class="shop__secondary"]//text()'
            XPATH_REVIEW_TEXT_2 = './/div[@class="Ge5tnd pd-reviews__user-review-full"]//text()'
            #Comments DNE
            #XPATH_REVIEW_COMMENTS = './/span[@class="a-size-base"]//text()'
            XPATH_AUTHOR = './/span[contains(@class,"shop__secondary")]//text()'
            XPATH_REVIEW_TEXT_3 = './/div[contains(@id,"dpReviews")]/div/text()'
            #HELPFUL DNE
            #XPATH_HELPFUL_VOTE_DIV = './/div[@class="a-row a-spacing-small"]'
            XPATH_FROM = './/span[contains(@class,"e2q0ib")]//text()'
            
            raw_review_author = review.xpath(XPATH_AUTHOR)[1].split(' ')[0]
            raw_review_rating = review.xpath(XPATH_RATING)[0][0]
            raw_review_header = review.xpath(XPATH_REVIEW_HEADER)[0]
            raw_review_posted_date = review.xpath(XPATH_REVIEW_POSTED_DATE)[0]
            raw_review_text1 = review.xpath(XPATH_REVIEW_TEXT_2)
            raw_review_from = review.xpath(XPATH_FROM)
            

            try:
                review_posted_date = dateparser.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
            except:
                review_posted_date = None
            review_text = ' '.join(' '.join(raw_review_text1).split())

            review_dict = {

                                'review_text': review_text,
                                'review_posted_date': review_posted_date,
                                'review_header': raw_review_header,
                                'review_rating': raw_review_rating,
                                'review_author': raw_review_author,
                                'review_from': raw_review_from

                            }
            reviews_list.append(review_dict)

        data = {
                    'ratings': ratings_dict,
                    'reviews': reviews_list,
                    'url': google_url,
                    'name': product_name,
                    'price': product_price
                
                }
        return data

    return {"error": "failed to process the page", "url": google_url}
            

def ReadGsin():
    # Add your own GSINs here
    gsinList = [7008326304182003253]
    pg_amounts = 5
    extracted_data = []
    
    for gsin in gsinList:
        print("Downloading and processing page {0}".format(gsin))
        for i in range(1,pg_amounts+1):   
            extracted_data.append(ParseReviews(gsin,i))
            sleep(5)
    f = open('google.json', 'w')
    dump(extracted_data, f, indent=4)
    f.close()

if __name__ == '__main__':
    ReadGsin()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Librar